In [109]:
import polars as pl

from datetime import datetime


In [110]:
btc = pl.read_csv('P:/ECB business areas/DGI/IPA/Personal folders/Lopardo/Etherscan_transaction/data/counterfactuals/btc-usd-daily_open.csv')
eth_net = pl.read_csv('P:/ECB business areas/DGI/IPA/Personal folders/Lopardo/Etherscan_transaction/data/counterfactuals/eth_network_utilization.csv')
sp500 = pl.read_csv('P:/ECB business areas/DGI/IPA/Personal folders/Lopardo/Etherscan_transaction/data/counterfactuals/sp500_historical_data.csv')
us_yield = pl.read_csv('P:/ECB business areas/DGI/IPA/Personal folders/Lopardo/Etherscan_transaction/data/counterfactuals/us_2y_yield.csv')

In [111]:
start_date = datetime(2019, 1, 1)

In [112]:
us_yield = us_yield.with_columns(
        pl.col('observation_date').str.strptime(pl.Date).alias('Date')
    ).select(['Date', 'DGS2'])

us_yield.head()

Date,DGS2
date,f64
2019-01-02,2.5
2019-01-03,2.39
2019-01-04,2.5
2019-01-07,2.53
2019-01-08,2.58


In [113]:
sp500 = sp500.with_columns(
    [
        pl.col('Date').str.strptime(pl.Date, format="%Y-%m-%d %H:%M:%S %Z"),
        pl.col('Open').alias('SP500')
    ]
).select(['Date', 'SP500'])


sp500.head()

Date,SP500
date,f64
2019-01-02,2476.959961
2019-01-03,2491.919922
2019-01-04,2474.330078
2019-01-07,2535.610107
2019-01-08,2568.110107


In [114]:
eth_net = eth_net.with_columns(
    [
        pl.col('Date(UTC)').str.strptime(pl.Date, format='%m/%d/%Y').alias('Date'), 
        pl.col('Value').alias('eth_utilization')
    ]
).select(['Date', 'eth_utilization'])


eth_net.head()

Date,eth_utilization
date,f64
2015-07-30,0.0
2015-07-31,0.0
2015-08-01,0.0
2015-08-02,0.0
2015-08-03,0.0


In [115]:
btc = btc.with_columns(
    [
        pl.col('snapped_at').str.strptime(pl.Date, format="%Y-%m-%d %H:%M:%S %Z").alias('Date'),
        pl.col('price').alias('BTC')
    ]  
).select(['Date', 'BTC'])

btc.head()

Date,BTC
date,f64
2013-04-28,135.3
2013-04-29,141.96
2013-04-30,135.3
2013-05-01,117.0
2013-05-02,103.43


In [116]:
df = btc.join(
        eth_net, 
        on='Date'
    ).join(
        sp500, 
        on='Date'
    ).join(
        us_yield, 
        on='Date'
    )

df.head()

Date,BTC,eth_utilization,SP500,DGS2
date,f64,f64,f64,f64
2019-01-02,3794.264254,0.7468,2476.959961,2.5
2019-01-03,3872.172184,0.7613,2491.919922,2.39
2019-01-04,3776.525808,0.7607,2474.330078,2.5
2019-01-07,4013.520091,0.7115,2535.610107,2.53
2019-01-08,3989.668718,0.7011,2568.110107,2.58


In [118]:
df.write_csv('counterfactuals.csv')

NameError: name 'eth_usd' is not defined